In [1]:
import torch
import json

In [2]:
import os
print(os.getcwd())
# Please change the root to an absolute or relative path to DomiKnowS root.
# In case relative path is used, consider the printed `CWD` as current working directory.
root = '/home/hfaghihi/Framework/DomiKnowS'
import sys
sys.path.append(root)
from typing import Any, Dict

/home/hfaghihi/Framework/DomiKnowS/examples/Propara


In [3]:
with open("updated_test_data.json", "r") as file:
    data = json.load(file)
data[2]

{'id': '37',
 'entities': ['bones'],
 'steps': ['A plant of animal dies in a watery environment.',
  'Is buried in mud and silt.',
  'Soft tissues quickly decompose leaving behind hard bones or shells.',
  'Over time sediment builds over the top and hardens into rock.',
  'As the bone decays mineral seeps in replacing it.',
  'Fossils are formed.'],
 'entity_step': [[0.005079575348645449,
   0.003536409232765436,
   0.9913840293884277],
  [0.0023296682629734278, 0.002440220210701227, 0.9952301979064941],
  [0.004893037490546703, 0.004029534757137299, 0.9910774230957031],
  [0.9432244300842285, 0.05380963161587715, 0.002965932944789529],
  [0.9029137492179871, 0.09116195142269135, 0.005924322176724672],
  [0.005038038361817598, 0.008660664781928062, 0.9863013029098511],
  [0.006529272999614477, 0.009216826409101486, 0.9842538833618164]]}

In [4]:
from regr.data.reader import RegrReader

In [5]:
class ProparaReader(RegrReader):
    def parse_file(self):
        with open(self.file, 'r') as f:
            lines = []
            for line in f:
                try:
                    if line != "\n":
                        lines.append(json.loads(str(line)))
                except:
                    raise
        items = lines
        final_dict = []
        for item in items:
            for i in range(len(item['participants'])):
                instance = item.copy()
                instance['participants'] = [item['participants'][i]]
                instance['states'] = item['states'][i]
                final_dict.append(instance)
                
        return final_dict
    
#     def getDataval(self, item):
#         return item
                
    def getParaIDval(self, item):
        return [item['para_id']]
    
    def getSentencesval(self, item):
        data = ['step 0 goes here']
        data.extend(item['sentence_texts'])
        return data
    
    def getEntityval(self, item):
        return item['participants']
    
    def getnon_existenceval(self, item):
        values = []
        for value in item['states']:
            if value == "-":
                values.append(1)
            else:
                values.append(0)
        return values
    
    def getunkowneval(self, item):
        values = []
        for value in item['states']:
            if value == "?":
                values.append(1)
            else:
                values.append(0)
        return values
    
    def getlocationeval(self, item):
        values = []
        for value in item['states']:
            if value != "?" and value != "-":
                values.append(1)
            else:
                values.append(0)
        return values
    
    def getLocationTextval(self, item):
        values = []
        for value in item['states']:
            if value != "?" and value != "-":
                values.append(value)
            else:
                values.append("NAN")
        return values
    
    
    
#     def getstepsval(self, item):
#         num_steps = len(item['steps']) + 1
#         rel = torch.ones(num_steps,1)
#         sentences = ["step 0 information"]
#         sentences.extend(item['steps'])
#         return rel, sentences
    
#     def getnon_existenceval(self, item):
#         values = []
#         for step in range(len(item['steps']) + 1):
#             values.append([1 - item['entity_step'][step][2], item['entity_step'][step][2]])
#         return torch.tensor(values)
            
#     def getknownval(self, item):
#         values = []
#         for step in range(len(item['steps']) + 1):
#             values.append([1 - item['entity_step'][step][0], item['entity_step'][step][0]])
#         return torch.tensor(values)
    
#     def getunknownval(self, item):
#         values = []
#         for step in range(len(item['steps']) + 1):
#             values.append([1 - item['entity_step'][step][1], item['entity_step'][step][1]])
#         return torch.tensor(values)
    
#     def getactionval(self, item):
#         action1s = torch.diag(torch.ones(len(item['steps']) + 1) )[:-1]
#         action2s = torch.diag(torch.ones(len(item['steps']) + 1) )[1:]
#         raw = torch.zeros(len(item['steps']))
#         return action1s, action2s
    
#     def getcreateval(self, item):
#         actions = []
#         prev_state = item['entity_step'][0]
#         for sid, step in enumerate(item['steps']):
#             o = 0
#             c = 0
#             d = 0
#             o += (prev_state[0] * item['entity_step'][sid+1][0])
#             o += (prev_state[0] * item['entity_step'][sid+1][1])
#             o += (prev_state[1] * item['entity_step'][sid+1][0])
#             o += (prev_state[1] * item['entity_step'][sid+1][1])
#             o += (prev_state[2] * item['entity_step'][sid+1][2])
#             d += (prev_state[0] * item['entity_step'][sid+1][2])
#             d += (prev_state[1] * item['entity_step'][sid+1][2])
#             c += (prev_state[2] * item['entity_step'][sid+1][1])
#             c += (prev_state[2] * item['entity_step'][sid+1][0])
#             actions.append([1-c, c])
#             prev_state = item['entity_step'][sid+1]
        
#         return torch.tensor(actions)
                    
#     def getdestroyval(self, item):
#         actions = []
#         prev_state = item['entity_step'][0]
#         for sid, step in enumerate(item['steps']):
#             o = 0
#             c = 0
#             d = 0
#             o += (prev_state[0] * item['entity_step'][sid+1][0])
#             o += (prev_state[0] * item['entity_step'][sid+1][1])
#             o += (prev_state[1] * item['entity_step'][sid+1][0])
#             o += (prev_state[1] * item['entity_step'][sid+1][1])
#             o += (prev_state[2] * item['entity_step'][sid+1][2])
#             d += (prev_state[0] * item['entity_step'][sid+1][2])
#             d += (prev_state[1] * item['entity_step'][sid+1][2])
#             c += (prev_state[2] * item['entity_step'][sid+1][1])
#             c += (prev_state[2] * item['entity_step'][sid+1][0])
#             actions.append([1-d, d])
#             prev_state = item['entity_step'][sid+1]
#         return torch.tensor(actions)
    
#     def getotherval(self, item):
#         actions = []
#         prev_state = item['entity_step'][0]
#         for sid, step in enumerate(item['steps']):
#             o = 0
#             c = 0
#             d = 0
#             o += (prev_state[0] * item['entity_step'][sid+1][0])
#             o += (prev_state[0] * item['entity_step'][sid+1][1])
#             o += (prev_state[1] * item['entity_step'][sid+1][0])
#             o += (prev_state[1] * item['entity_step'][sid+1][1])
#             o += (prev_state[2] * item['entity_step'][sid+1][2])
#             d += (prev_state[0] * item['entity_step'][sid+1][2])
#             d += (prev_state[1] * item['entity_step'][sid+1][2])
#             c += (prev_state[2] * item['entity_step'][sid+1][1])
#             c += (prev_state[2] * item['entity_step'][sid+1][0])
#             actions.append([1-o, o])
#             prev_state = item['entity_step'][sid+1]
#         return torch.tensor(actions)
    
#     def getbeforeval(self, item):
#         b1s = []
#         b2s = []
#         for step in range(len(item['steps']) + 1):
#             b1 = torch.zeros(len(item['steps']) + 1)
#             b1[step] = 1
#             for step1 in range(len(item['steps']) + 1):
#                 b2 = torch.zeros(len(item['steps']) + 1)
#                 b2[step1] = 1
#                 b1s.append(b1)
#                 b2s.append(b2)
#         return torch.stack(b1s), torch.stack(b2s)
    
#     def getbefore_trueval(self, item):
#         num_steps = len(item['steps']) + 1
#         values = torch.zeros(num_steps * num_steps)
#         for step in range(len(item['steps']) + 1):
#             for step1 in range(step + 1, len(item['steps']) + 1):
#                 values[(step*num_steps)+step1] = 1
#         return values
    

In [6]:
ReaderObjectsIterator = ProparaReader("emnlp18/grids.v1.train.json", 'parse')
list(iter(ReaderObjectsIterator))[0]

{'Entity': ['magma'],
 'LocationText': ['deep in the earth',
  'deep in the earth',
  'volcano',
  'volcano',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN'],
 'ParaID': ['7'],
 'Sentences': ['step 0 goes here',
  'Magma rises from deep in the earth.',
  'The magma goes into volcanos.',
  'The volcanos pressure the magma upwards.',
  'The pressure causes the magma to push through the surface of the volcano.',
  'The lava cools.',
  'The lava forms new rock.',
  'New magma is pressured to the surface of the volcano.',
  'The volcano bursts through the rock the formed after the last eruption.'],
 'locatione': [1, 1, 1, 1, 0, 0, 0, 0, 0],
 'non_existence': [0, 0, 0, 0, 1, 1, 1, 1, 1],
 'unkowne': [0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [7]:
from regr.graph import Graph, Concept, Relation
from regr.graph.logicalConstrain import orL, andL, existsL, notL, atLeastL, atMostL, ifL, nandL, eqL

Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
    procedure = Concept("procedure")
    text = Concept("text")
    entity = Concept("entity")
    (procedure_text, procedure_entity) = procedure.has_a(arg1=text, arg2=entity)
    step = Concept("step")
    (text_contain_step, ) = text.contains(step)
    
    pair = Concept("pair")
    (pair_entity, pair_step) = pair.has_a(entity, step)
    
    word = Concept("word")
    (pair_contains_words, ) = pair.contains(word)
    
    non_existence = pair("non_existence")
    unknown_loc = pair("unknown_location")
    known_loc = pair("known_location")
    
    triplet = Concept("triplet")
    (triplet_entity, triplet_step, triplet_word) = triplet.has_a(entity, step, word)
    
    before = Concept("before")
    (before_arg1, before_arg2) = before.has_a(arg1=step, arg2=step)
    
#     action = Concept("action")
#     (action_arg1, action_arg2) = action.has_a(arg1=step, arg2=step)
#     create = action(name="create")
#     destroy = action(name="destroy")
#     other = action(name="other")
    
    #LC5 : If action is create then the first step should be non_existence and the second step can be either known_loc or unknown_loc
#     ifL(create, ("x", "y", ), andL(non_existence, ("x", ), orL(known_loc, ("y", ), unknown_loc, ("y", ))))
    
#     #LC 6 : If action is destroy, then first step should be either known_loc,or unknown_loc and the next step should be non_existence 
#     ifL(destroy, ("x", "y", ), andL(orL(known_loc, ("x", ), unknown_loc, ("x", )), non_existence, ("y", )))
    
#     #LC7 : There should be at most 1 create
#     atMostL(1, ("x", ), create, ("x", ))
    
#     #LC8 : There should be at most one destroy
#     atMostL(1, ("x", ), destroy, ("x", ))
    
#     #LC9 : If (x1,x2) is create and (y1, y2) is destroy, then the pair(x2,y2) in before should have the property "check" equal to 1.
#     # I will have to check if this eqL works if not will update it
#     ifL(andL(create, ("x1", "x2"), destroy, ("y1", "y2")), eqL(before, "check", 1), ("x2", "y2"))
    
#     #LC1 : An action can not be create, destroy and other at the same time
#     nandL(create, destroy, other)
    
#     #LC2 : An action should at least be one of the create, destroy or other
#     orL(create, destroy, other)
    
#     #LC3 : A step can not be known_loc, unknown_loc and non_existence at the same time
#     nandL(known_loc, unknown_loc, non_existence)
    
#     #LC4 : A step should at least be one of known_loc, unknown_loc or non_existence
#     orL(known_loc, unknown_loc, non_existence)
    




Log file for dataNode is in: /home/hfaghihi/Framework/DomiKnowS/examples/Propara/datanode.log


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'text'), ('arg2', 'entity')]) is used.
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'step'), ('arg2', 'step')]) is used.


In [8]:
from regr.sensor.pytorch.sensors import ReaderSensor, JointSensor
from regr.sensor.pytorch.relation_sensors import EdgeSensor

class EdgeReaderSensor(EdgeSensor, ReaderSensor):
    def __init__(self, *pres, relation, mode="forward", keyword=None, **kwargs):
        super().__init__(*pres, relation=relation, mode=mode, **kwargs)
        self.keyword = keyword
        self.data = None
        
# class JoinReaderSensor(JointSensor, ReaderSensor):
#     pass
            
# class JoinEdgeReaderSensor(JointSensor, EdgeReaderSensor):
#     pass

class JoinReaderSensor(JointSensor, ReaderSensor):
    pass

class JoinEdgeReaderSensor(JoinReaderSensor, EdgeSensor):
    pass

In [24]:
from regr.sensor.pytorch.sensors import ReaderSensor, FunctionalSensor, JointSensor
from regr.sensor.pytorch.learners import TorchLearner, ModuleLearner
from regr.program import LearningBasedProgram
from regr.program.model.pytorch import PoiModel
import torch
from torch import nn

def model_declaration():

    graph.detach()

    procedure['id'] = ReaderSensor(keyword='ParaID')
    entity['raw'] = ReaderSensor(keyword='Entity')
    text['raw'] = ReaderSensor(keyword="Sentences")
    
    def sentence_parser(text):
        sentence = ""
        for item in text[:-1]:
            sentence += str(item) + "</s>"
        sentence += str(text[-1])
        return [sentence]
    
    text['ready'] = FunctionalSensor(text['raw'], forward=sentence_parser)
    
    def sentence_separator(text):
        mapping = torch.ones(len(text), 1)
        return mapping, text
    
    step[text_contain_step, 'raw'] = JointSensor(text['raw'], forward=sentence_separator)
    
    
    def procedure_candidate(inputs):
        mapping1 = torch.zeros(len(inputs[0])*len(inputs[2]), len(inputs[0]))
        for i in range(len(inputs[0])):
            mapping1[i*len(inputs[2]):(i+1)*len(inputs[2]), i] = 1
        
        mapping2 = torch.zeros(len(inputs[2]) * len(inputs[0]), len(inputs[2]))
        
        for i in range(len(inputs[2])):
            mapping2[i*len(inputs[0]):(i+1)*len(inputs[0]), i] = 1
            
        text = ["Where is " + str(inputs[0][0]) + "?!</s>" + str(inputs[1])] * len(inputs[2])
        return mapping1, mapping2, text
    
    pair[pair_entity.reversed, pair_step.reversed, 'text'] = JointSensor(entity['raw'], text['ready'], step['raw'], forward=procedure_candidate)
    
    
    class RoBertaTokenizorSensor(JointSensor):
        from transformers import RobertaTokenizerFast
        import functools
        import operator
        TRANSFORMER_MODEL = 'roberta-large'
        tokenizer = RobertaTokenizerFast.from_pretrained(TRANSFORMER_MODEL)

        def roberta_extract_timestamp_sequence(inputs, end_time):
            f_out = []
            padding = 0
            for time in range(-1, end_time - 1):
                timestamp_id = []
                if time == -1:
                    check = -1
                    for index, ids in enumerate(inputs['input_ids'][time + 1]):
                        if ids == 2:
                            check += 1
                            if check == 0:
                                padding = index + 1
                        if check == -1:
                            timestamp_id.append(0)
                        elif ids == 2:
                            timestamp_id.append(0)
                        else:
                            timestamp_id.append(2)
                else:
                    check = -1
                    for index, ids in enumerate(inputs['input_ids'][time + 1]):
                        if ids == 2:
                            check += 1
                        if check == -1:
                            timestamp_id.append(0)
                        elif ids == 2:
                            timestamp_id.append(0)
                        else:
                            if check < time :
                                timestamp_id.append(1)
                            elif check == time:
                                timestamp_id.append(2)
                            else:
                                timestamp_id.append(3)
                timestamp_id = torch.tensor(timestamp_id).to(device=inputs['input_ids'].device)
                f_out.append(timestamp_id)
            inputs['timestep_type_ids'] = torch.stack(f_out)
            return inputs, padding

        def forward(self, inputs):
            sentences = inputs[0]
            tokens = self.tokenizer(
                sentences,
                return_tensors='pt',
                return_offsets_mapping=True,
            )
            token_strings = []
            token_nums = []
            mapping = torch.zeros(len(tokens['input_ids'][0])*len(sentences), len(sentences))
            tokens, padding = roberta_extract_timestamp_sequence(tokens, end_time=len(sentences))
            padding = [padding - 1] * len(sentences)
            for sen_num in range(len(sentences)):
                token_strings.append(self.tokenizer.convert_ids_to_tokens(tokens['input_ids'][sen_num]))
                token_nums.append(len(tokens['input_ids'][sen_num]))
                mapping[sen_num*len(tokens['input_ids'][0]):((sen_num+1)*len(tokens['input_ids'][0])),sen_num] = 1

            for key in tokens.keys():
                tokens[key] = functools.reduce(operator.iconcat, tokens[key], [])
            tokens['tokens'] = token_strings
            tokens['token_nums'] = token_nums

            return mapping, list(tokens.values()), padding
        

    word[pair_contains_words, 'input_ids', 'attention_mask', 'offset_mapping', "timestep_type_ids", 'tokens', 'token_nums', "padding"] = RoBertaTokenizorSensor(pair['text'], pair_entity.reversed, pair_step.reversed)
    
    pair[pair_contains_words.reversed] = FunctionalSensor(word[pair_contains_words], forward=lambda x : x[0].t)
    
    class BatchifyLearner(TorchLearner):
        import functools
        import operator
        def __init__(self, *pres, batchify=True, **kwargs):
            super().__init__(*pres, **kwargs)
            self.batchify = batchify
            
        def define_inputs(self):
            self.inputs = []
            if len(self.batchify):
                hinter = self.fetch_value(self, self.batchify[0])
            for pre in self.pres:
                values = self.fetch_value(pre)
                if len(self.batchify):
                    final_val = []
                    for hint in hinter.t():
                        slicer = torch.nonzero(hint).squeeze(-1)
                        final_val.append(values.index_select(0, slicer))
                    values = torch.stack(final_val)
                self.inputs.append(values)
                
        def update_pre_context(
            self,
            data_item: Dict[str, Any],
            concept=None
        ) -> Any:
            super().update_pre_context(data_item, concept)
            concept = concept or self.concept
            for batchifier in self.batchify:
                for sensor in concept[batchifier].find(self.non_label_sensor):
                    sensor(data_item=data_item)
                    
        def update_context(
            self,
            data_item: Dict[str, Any],
            force=False,
            override=True):
            if not force and self in data_item:
                # data_item cached results by sensor name. override if forced recalc is needed
                val = data_item[self]
            else:
                self.update_pre_context(data_item)
                self.define_inputs()
                val = self.forward_wrap()
                
                if len(self.batchify):
                    val = functools.reduce(operator.iconcat, val, [])
                    
                data_item[self] = val
            if override and not self.label:
                data_item[self.prop] = val  # override state under property name
                
           
    class BatchifyModuleLearner(ModuleLearner, BatchifyLearner):
        pass
    
    class RobertaModelLearner(BatchifyModuleLearner):
        device="cpu"
        def forward(self, inputs):
            running = {}
            running["input_ids"] = inputs[0]
            running["attention_mask"] = inputs[1]
            running["timestep_type_ids"] = inputs[2]
            transformer_result = self.model(**running)
            return transformer_result[0]
        
    from roberta import RobertaModel
    word["embeding"] = RobertaModelLearner('input_ids', 'attention_mask', 'offset_mapping', batchify=[pair_contains_words], module=RobertaModel.from_pretrained('tli8hf/unqover-roberta-large-squad'))
            
    
    import torch.nn as nn
    
    word['start'] = BatchifyModuleLearner('embedding', batchify=[pair_contains_words], module=nn.Sequential(nn.Linear(768, 1), nn.Softmax(dim=-1)))
    word['end'] = BatchifyModuleLearner('embedding', batchify=[pair_contains_words], module=nn.Sequential(nn.Linear(768, 1), nn.Softmax(dim=-1)))
    # word[step_contains_word, 'raw'] = ReaderSensor(keyword='words')
#     entity['raw'] = ReaderSensor(keyword='entities')

#     step[non_existence] = ReaderSensor(procedure_contain_step.forward, 'text', keyword='non_existence')
#     step[unknown_loc] = ReaderSensor(procedure_contain_step.forward, 'text', keyword='unknown')
#     step[known_loc] = ReaderSensor(procedure_contain_step.forward, 'text', keyword='known')
    
#     step[non_existence] = ReaderSensor(keyword='non_existence', label=True)
#     step[unknown_loc] = ReaderSensor(keyword='unknown', label=True)
#     step[known_loc] = ReaderSensor(keyword='known', label=True)
    
#     action[action_arg1.backward, action_arg2.backward] = JoinReaderSensor(step['text'], keyword='action')
    
#     action[create] = ReaderSensor(action_arg1.backward, action_arg2.backward, keyword='create')
#     action[destroy] = ReaderSensor(action_arg1.backward, action_arg2.backward, keyword='destroy')
#     action[other] = ReaderSensor(action_arg1.backward, action_arg2.backward, keyword='other')
    
#     action[create] = ReaderSensor(keyword='create', label=True)
#     action[destroy] = ReaderSensor(keyword='destroy', label=True)
#     action[other] = ReaderSensor(keyword='other', label=True)
    
#     before[before_arg1.backward, before_arg2.backward] = JoinReaderSensor(step['text'], keyword="before")
    
#     before["check"] = ReaderSensor(before_arg1.backward, before_arg2.backward, keyword="before_true")
#     before["check"] = ReaderSensor(keyword="before_true", label=True)
    
    program = LearningBasedProgram(graph, **{
        'Model': PoiModel,
        'poi': (pair['text'], ),
        'loss': None,
        'metric': None,
    })
    return program
#     return graph

In [25]:
def main():
    # set logger level to see training and testing logs
    import logging
    logging.basicConfig(level=logging.INFO)

    lbp = model_declaration()

    dataset = ProparaReader("emnlp18/grids.v1.train.json", 'parse')  # Adding the info on the reader

    for datanode in lbp.populate(dataset, device="cpu"):
        print(datanode)

In [26]:
main()

Some weights of RobertaModel were not initialized from the model checkpoint at tli8hf/unqover-roberta-large-squad and are newly initialized: ['roberta.embeddings.timestep_type_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error during updating data item with sensor readersensor-4
Error during updating data item with sensor jointsensor-1
Error during updating data item with sensor functionalsensor-2


AttributeError: 'str' object has no attribute 'item'

In [11]:
from regr.graph import DataNodeBuilder
from regr.sensor.sensor import Sensor
import logging
logging.basicConfig(level=logging.INFO)

graph1 = model_declaration()
# dataset = ProparaReader("emnlp18/grids.v1.train.json", 'parse')
# for datanode in lbp.populate(list(dataset)[0:1], device="cpu"):
#     pass
dataset = ProparaReader("emnlp18/grids.v1.train.json", 'parse')
for data in list(iter(dataset))[0:1]:
    context = {"graph": graph1}
    context.update(data)
    data_item = DataNodeBuilder(context)
    # data_item
    for sensor in entity['raw'].find(Sensor):
        sensor(data_item)
        
    datanode = data_item.getDataNode()
    print(data_item)
#     datanode.inferILPConstrains(create, destroy, other, non_existence, known_loc, unknown_loc, fun=None)

Some weights of RobertaModel were not initialized from the model checkpoint at tli8hf/unqover-roberta-large-squad and are newly initialized: ['roberta.embeddings.timestep_type_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Entity
None
{'graph': Graph(name='global', fullname='global'), 'Entity': ['magma'], 'LocationText': ['deep in the earth', 'deep in the earth', 'volcano', 'volcano', 'NAN', 'NAN', 'NAN', 'NAN', 'NAN'], 'ParaID': ['7'], 'Sentences': ['step 0 goes here', 'Magma rises from deep in the earth.', 'The magma goes into volcanos.', 'The volcanos pressure the magma upwards.', 'The pressure causes the magma to push through the surface of the volcano.', 'The lava cools.', 'The lava forms new rock.', 'New magma is pressured to the surface of the volcano.', 'The volcano bursts through the rock the formed after the last eruption.'], 'locatione': [1, 1, 1, 1, 0, 0, 0, 0, 0], 'non_existence': [0, 0, 0, 0, 1, 1, 1, 1, 1], 'unkowne': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'Counter_setitem': 2, 'Counter/global/entity/raw/readersensor-1': {None: {'counter': 1, 'recent': True}}, ReaderSensor(name='readersensor-1', fullname='global/entity/raw/readersensor-1'): None, Property(name='raw', fullname='global/entity/raw'): N

In [47]:
all_updates[2]

{'id': '37',
 'entities': ['bones'],
 'steps': [(tensor([1.], device='cuda:0', dtype=torch.float64),
   tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([0.], device='cuda:0', dtype=torch.float64)),
  (tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([1.], device='cuda:0', dtype=torch.float64)),
  (tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([1.], device='cuda:0', dtype=torch.float64)),
  (tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([1.], device='cuda:0', dtype=torch.float64)),
  (tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([1.], device='cuda:0', dtype=torch.float64)),
  (tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([0.], device='cuda:0', dtype=torch.float64),
   tensor([1.]

In [59]:
context = {"graph": graph}
context.update(list(iter(dataset))[2])
data_item = DataNodeBuilder(context)
# data_item
for sensor in procedure['id'].find(Sensor):
    sensor(data_item)
for sensor in step['text'].find(Sensor):
    sensor(data_item)
for sensor in step[procedure_contain_step.forward].find(Sensor):
    sensor(data_item)
for sensor in step[known_loc].find(Sensor):
    sensor(data_item)
for sensor in step[unknown_loc].find(Sensor):
    sensor(data_item)
for sensor in step[non_existence].find(Sensor):
    sensor1 = sensor
    sensor(data_item)
for sensor in action[action_arg1.backward].find(Sensor):
    sensor(data_item)
for sensor in action[action_arg2.backward].find(Sensor):
    sensor(data_item)
for sensor in action[destroy].find(Sensor):
    sensor(data_item)
for sensor in action[create].find(Sensor):
    sensor(data_item)
for sensor in action[other].find(Sensor):
    sensor(data_item)
print(data_item[sensor1])
print(list(iter(dataset))[2]['non_existence'])
datanode = data_item.getDataNode()
print(datanode.findDatanodes(select=step)[1].getAttribute('non_existence'))

tensor([[0.9976, 0.0024],
        [0.0150, 0.9850],
        [0.0084, 0.9916],
        [0.0046, 0.9954],
        [0.0042, 0.9958],
        [0.0043, 0.9957],
        [0.0049, 0.9951]])
tensor([[0.0086, 0.9914],
        [0.0048, 0.9952],
        [0.0089, 0.9911],
        [0.9970, 0.0030],
        [0.9941, 0.0059],
        [0.0137, 0.9863],
        [0.0157, 0.9843]])
tensor([0.0150, 0.9850])


In [56]:
data_item

{'graph': Graph(name='global', fullname='global'),
 'action': (tensor([[1., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0.]]),
  tensor([[0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0., 1.]])),
 'before_true': tensor([0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
         1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 'before': (tensor([[1., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.],
    

In [52]:
list(iter(dataset))[2]

{'action': (tensor([[1., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0.]]),
  tensor([[0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0., 1.]])),
 'before_true': tensor([0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
         1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 'before': (tensor([[1., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0., 0., 0.],
          [1., 

In [22]:
import torch
tt = torch.randn(10, 12).to(1)
tt.device.index

1